In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm

Criando o dataset

In [ ]:
data = {
    'price': [1.99, 1.49, 2.99, 3.99, 1.99, 2.49, 3.99, 2.99, 1.99, 0.99],
    'measurement': [0.2, 0.4, 0.1, 0.1, 0.2, 0.1, 0.1, 0.1, 0.2, 0.5],
    'weight': [2, 4, 1, 1, 2, 1, 1, 1, 2, 5],
    'units_sold': [1, 2, 1, 0, 2, 0, 0, 0, 1, 4]
}

df = pd.DataFrame(data)


Separando as variáveis

In [ ]:
X = df[["price", "measurement", "weight"]]
y = df[["units_sold"]]

Treinando o GLM

In [ ]:


mod = sm.GLM(y, X, family=sm.families.Poisson(link=sm.families.links.log()))
res = mod.fit()
display(res.summary())


/usr/local/lib/python3.10/dist-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:             units_sold   No. Observations:                   10
Model:                            GLM   Df Residuals:                        8
Model Family:                 Poisson   Df Model:                            1
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -9.5168
Date:                Sun, 07 Jan 2024   Deviance:                       4.5404
Time:                        11:21:13   Pearson chi2:                     4.02
No. Iterations:                     5   Pseudo R-squ. (CS):             0.6321
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
price          -0.4604      0.223     -2.061      0.039      -0.898      -0.023
measurement     0.0375      0.011      3.493      0.000       0.016       0.059
weight          0.3749      0.107      3.493      0.000       0.165       0.585
===============================================================================
"""

A título de comparação, vamos treinar esse modelo em  um OLS (regressão linear)

In [ ]:
mod2 = sm.OLS(y, X)
res2 = mod2.fit()
display(res2.summary())

/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             units_sold   R-squared (uncentered):                   0.917
Model:                            OLS   Adj. R-squared (uncentered):              0.897
Method:                 Least Squares   F-statistic:                              44.33
Date:                Sun, 07 Jan 2024   Prob (F-statistic):                    4.69e-05
Time:                        11:21:28   Log-Likelihood:                         -6.6966
No. Observations:                  10   AIC:                                      17.39
Df Residuals:                       8   BIC:                                      18.00
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
price          -0.1618      0.079     -2.040      0.076      -0.345       0.021
measurement     0.0740      0.009      8.516      0.000       0.054       0.094
weight          0.7402      0.087      8.516      0.000       0.540       0.941
==============================================================================
Omnibus:                        0.712   Durbin-Watson:                   2.651
Prob(Omnibus):                  0.700   Jarque-Bera (JB):                0.614
Skew:                           0.469   Prob(JB):                        0.736
Kurtosis:                       2.230   Cond. No.                     1.08e+17
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The smallest eigenvalue is 9.03e-33. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

Agora, vamos aplicar os modelos treinados em um conjunto de teste de dados nunca visto antes - essa prática é comum quanto estamos mais interessados na predição do que na explicação das variáveis - isso é um assunto mais comum de machine learning.

Por hora, vamos nos concentrar no fato de que nosso objetivo com essas regressões é prever o unit_sold, e temos 2 modelos treinados - um linear e um OLM. Agora, temos preços e tamanhos de outros produtos e queremos prever quanto teríamos de unit_sold

In [ ]:
# nossos novos dados, os quais não sabemos o unit sold (queremos prever isso)
data2 = {
    'price': [0.99, 2.49, 1.49, 1.99, 4.99],
    'measurement': [0.4, 0.1, 0.3, 0.2, 0.1],
    'weight': [4, 1, 3, 2, 1],
}

df_test = pd.DataFrame(data2)

In [ ]:
X_test = df_test[["price", "measurement", "weight"]]

Vamos ver as predições no modelo GLM


In [ ]:
y_test_pred_glm = res.predict(X_test)
print(y_test_pred_glm)

0    2.883304
1    0.464031
2    1.568348
3    0.853090
4    0.146767
dtype: float64


Nossas previsões parecem bastante razoáveis. Como todos esses dados são amostrais inventados, é difícil dizer se nosso modelo está indo bem ou não. No entanto, todos os valores são positivos e seguem, até certo ponto, os nossos dados de amostra.
Ser positivo é importante aqui, uma vez que unit sold nunca será negativo (é quantidade vendida).

O fato de ser um número contínuo não é preocupante, uma vez que esses dados são inventados, e podemos aproxima-los para o inteiro mais próximo

Agora vamos testar na regressão linear

In [ ]:
y_test_pred_linear = res2.predict(X_test)
print(y_test_pred_linear)

0    2.830127
1    0.344628
2    2.001628
3    1.173128
4   -0.059946
dtype: float64


As previsões não parecem tão ruins, mas repare que uma das previsões acaba sendo negativa! Em nosso exemplo de caso de uso, nunca devemos ter valores negativos. Vemos uma previsão negativa porque o modelo de regressão linear assume uma distribuição gaussiana como seu domínio de valor, o que significa que os valores podem ser qualquer valor negativo ou positivo. Portanto, este tipo de modelo permite previsões de valores negativos devido a esta suposição gaussiana.